# Kiểm tra GPU

In [1]:
!nvidia-smi

Sun May 12 20:32:20 2024       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 517.70       Driver Version: 517.70       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   47C    P0    24W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Cài đặt và import các thư viện cần thiết

In [2]:
!pip install transformers datasets

In [3]:
pip install accelerate -U

Note: you may need to restart the kernel to use updated packages.


ERROR: Cannot uninstall 'TBB'. It is a distutils installed project and thus we cannot accurately determine which files belong to it which would lead to only a partial uninstall.


  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/e9/bb/1edd2c836071e91d2bd331b9542bbd592e23d1474645b9c6fd56232caace/accelerate-0.30.1-py3-none-any.whl.metadata
  Obtaining dependency information for torch>=1.10.0 from https://files.pythonhosted.org/packages/2a/b7/a3cf5fd40334b9785cc83ee0c96b50603026eb3aa70210a33729018e7029/torch-2.3.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for typing-extensions>=4.8.0 from https://files.pythonhosted.org/packages/01/f3/936e209267d6ef7510322191003885de524fc48d1b43269810cd589ceaf5/typing_extensions-4.11.0-py3-none-any.whl.metadata
  Obtaining dependency information for mkl<=2021.4.0,>=2021.1.1 from https://files.pythonhosted.org/packages/fe/1c/5f6dbf18e8b73e0a5472466f0ea8d48ce9efae39bd2ff38cebf8dce61259/mkl-2021.4.0-py2.py3-none-win_amd64.whl.metadata
  Obtaining dependency information for intel-openmp==2021.* from https://files.pythonhosted.org/packages/6f/21/b590c0cc3888b24f2ac

# Restart runtime rồi chạy code phía dưới

In [4]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np

# Load model và tokenizer

In [5]:
model = AutoModelForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=2)
tokenizer = AutoTokenizer.from_pretrained("FPTAI/vibert-base-cased")

ImportError: 
AutoModelForSequenceClassification requires the PyTorch library but it was not found in your environment.
However, we were able to find a TensorFlow installation. TensorFlow classes begin
with "TF", but are otherwise identically named to our PyTorch classes. This
means that the TF equivalent of the class you tried to import would be "TFAutoModelForSequenceClassification".
If you want to use TensorFlow, please use TF classes instead!

If you really do want to use PyTorch please go to
https://pytorch.org/get-started/locally/ and follow the instructions that
match your environment.


# Dataset

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

df1_train = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/train_victsd.csv')
df1_valid = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/dev_victsd.csv')
df1_test = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/test_victsd.csv')

df2_train = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/train_vihsd.csv')
df2_valid = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/dev_vihsd.csv')
df2_test = pd.read_csv('/content/drive/MyDrive/Dataset_Reformat/test_vihsd.csv')

# # Nối các DataFrame theo chiều dọc (axis=0)
# concatenated_df1 = pd.concat([df1a, df1b], axis=0)

# # Nếu bạn muốn đặt lại chỉ mục của DataFrame sau khi nối
# concatenated_df1.reset_index(drop=True, inplace=True)

# # Nối các DataFrame theo chiều dọc (axis=0)
# concatenated_df2 = pd.concat([df2a, df2b], axis=0)

# # Nếu bạn muốn đặt lại chỉ mục của DataFrame sau khi nối
# concatenated_df2.reset_index(drop=True, inplace=True)

In [ ]:
df2_train.head()

,text,label
0,Em được làm fan cứng luôn rồi nè ❤️ reaction q...,0
1,Đúng là bọn mắt híp lò xo thụt :))) bên việt n...,2
2,Đậu Văn Cường giờ giống thằng sida hơn à,0
3,CÔN ĐỒ CỤC SÚC VÔ NHÂN TÍNH ĐỀ NGHI VN. NHÀ NƯ...,2
4,Từ lý thuyết đến thực hành là cả 1 câu chuyện ...,0


In [ ]:
def process_df2(df):
    df['label'] = df['label'].replace(2, 1)
    return df.dropna()

df2_train = process_df2(df2_train)
df2_valid = process_df2(df2_valid)
df2_test = process_df2(df2_test)

In [ ]:
df2_train.isna().sum()

text     0
label    0
dtype: int64

In [ ]:
def process_df1(df):
    return df.dropna()
df1_train = process_df1(df1_train)
df1_valid = process_df1(df1_valid)
df1_test = process_df1(df1_test)

In [ ]:
df1_valid.isna().sum()

text     0
label    0
dtype: int64

In [ ]:
tu_dien_viet_tat = {
    "a": "anh",
    "c": "chị",
    "e": "em",
    "b": "bạn",
    "bn": "bạn",
    "t": "tao",
    "m": "mày",
    "cmt": "bình luận",
    "ng": "người",
    "mn": "mọi người",
    "iu": "yêu",
    "ct": "chia tay",
    "ny": "người yêu",
    "vk": "vợ",
    "ck": "chồng",
    "vc": "vợ chồng",
    "ox": "ông xã",
    "bx": "bà xã",
    "h": "giờ",
    "g": "giờ",
    "ph": "phút",
    "bh": "bây giờ",
    "bjo": "bây giờ",
    "hn": "hôm nay",
    "hnay": "hôm nay",
    "hqua": "hôm qua",
    "htrc": "hôm trước",
    "nt": "nhắn tin",
    "nc": "nói chuyện",
    "mún": "muốn",
    "bit": "biết",
    "bik": "biết",
    "cb": "chuẩn bị",
    "nch": "nói chung",
    "dt": "điện thoại",
    "đt": "điện thoại",
    "bth": "bình thường",
    "j": "gì",
    "bn": "bao nhiêu",
    "ntn": "như thế nào",
    "k": "không",
    "ko": "không",
    "hok": "không",
    "hem": "không",
    "dc": "được",
    "đc": "được",
    "r": "rồi",
    "rùi": "rồi",
    "nhìu": "nhiều",
    "wá": "quá",
    "cg": "cũng",
    "uk": "ừ",
    "uh": "ừ",
    "ukm": "ừ",
    "G9": "ngủ ngon",
    "hic": "tiếng khóc",
    "vn": "Việt Nam"
}

In [ ]:
!pip install emoji_vietnamese

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 2.7 MB/s eta 0:00:00


In [ ]:
!pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 73.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 61.0 MB/s eta 0:00:00


In [ ]:
import underthesea
import string
import emoji_vietnamese

def preprocess_data(data):
    data = data.replace(
        r'http\S+', '').replace(r'www\S+', '')
    data = data.replace(
        '['+string.punctuation+']', '')
    data = data.lower().strip()
    data = emoji_vietnamese.demojize(data)
    new_str = data.split()
    for i in range(len(new_str)):
       if new_str[i] in tu_dien_viet_tat.keys():
         new_str[i] = tu_dien_viet_tat[new_str[i]]
    return underthesea.word_tokenize(" ".join(new_str), format='text')


In [ ]:
df1_train['text'] = df1_train['text'].map(preprocess_data)
df1_valid['text'] = df1_valid['text'].map(preprocess_data)
df1_test['text'] = df1_test['text'].map(preprocess_data)

df2_train['text'] = df2_train['text'].map(preprocess_data)
df2_valid['text'] = df2_valid['text'].map(preprocess_data)
df2_test['text'] = df2_test['text'].map(preprocess_data)

In [ ]:
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42, sampling_strategy=1)
X_resampled, y_resampled = ros.fit_resample(df1_train[['text']], df1_train['label'])
df1_train_new = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
df1_train_new.columns = ['text', 'label']

X_resampled, y_resampled = ros.fit_resample(df2_train[['text']], df2_train['label'])
df2_train_new = pd.concat([pd.DataFrame(X_resampled), pd.DataFrame(y_resampled)], axis=1)
df2_train_new.columns = ['text', 'label']

In [ ]:
from datasets import Dataset

def convert_to_datasets(df):
    dataset = Dataset.from_pandas(df).shuffle(seed=20)
    return dataset

train_dataset1 = convert_to_datasets(df1_train_new)
valid_dataset1 = convert_to_datasets(df1_valid)
test_dataset1 = convert_to_datasets(df1_test)

train_dataset2 = convert_to_datasets(df2_train_new)
valid_dataset2 = convert_to_datasets(df2_valid)
test_dataset2 = convert_to_datasets(df2_test)

In [ ]:
train_dataset1

Dataset({
    features: ['text', 'label'],
    num_rows: 12482
})

In [ ]:
# Tokenizer dataset và thêm padding
def tokenize_function(examples):
    return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=256)
tokenized_train_dataset1 = train_dataset1.map(tokenize_function, batched=True, remove_columns='text')
tokenized_valid_dataset1 = valid_dataset1.map(tokenize_function, batched=True, remove_columns='text')
tokenized_test_dataset1 = test_dataset1.map(tokenize_function, batched=True, remove_columns='text')

tokenized_train_dataset2 = train_dataset2.map(tokenize_function, batched=True, remove_columns='text')
tokenized_valid_dataset2 = valid_dataset2.map(tokenize_function, batched=True, remove_columns='text')
tokenized_test_dataset2 = test_dataset2.map(tokenize_function, batched=True, remove_columns='text')
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Map:   0%|          | 0/12482 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/39770 [00:00<?, ? examples/s]

Map:   0%|          | 0/2672 [00:00<?, ? examples/s]

Map:   0%|          | 0/6680 [00:00<?, ? examples/s]

# Training

In [ ]:
# Viết hàm tính accuracy
metric = load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-22-b113ca1a6718>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
# Cài đặt các tham số và train
training_args = TrainingArguments(
    output_dir="./results1",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer_1 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset1,
    eval_dataset=tokenized_valid_dataset1,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)



In [ ]:
trainer_1.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.360500,0.406511,0.871500
2,0.090200,0.714943,0.886000
3,0.021500,0.794023,0.888500


TrainOutput(global_step=2343, training_loss=0.15741163148770107, metrics={'train_runtime': 1670.539, 'train_samples_per_second': 22.416, 'train_steps_per_second': 1.403, 'total_flos': 4926228289505280.0, 'train_loss': 0.15741163148770107, 'epoch': 3.0})

In [ ]:
from sklearn.metrics import precision_recall_fscore_support

def compute_metrics_f1(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    # precision_recall_fscore_support returns a tuple containing precision, recall, f1, support respectively
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')

    return {
        'precision': precision,
        'recall': recall,
        'f1': f1,
    }


trainer_eval = Trainer(
    model=model,
    eval_dataset=tokenized_test_dataset1,
    compute_metrics=compute_metrics_f1,

)

trainer_eval.evaluate()


{'eval_loss': 0.424488365650177,
 'eval_precision': 0.42735042735042733,
 'eval_recall': 0.45454545454545453,
 'eval_f1': 0.44052863436123346,
 'eval_runtime': 15.3898,
 'eval_samples_per_second': 64.978,
 'eval_steps_per_second': 8.122}

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("FPTAI/vibert-base-cased", num_labels=2)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FPTAI/vibert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Cài đặt các tham số và train
training_args = TrainingArguments(
    output_dir="./results2",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)

trainer_2 = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset2,
    eval_dataset=tokenized_valid_dataset2,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    # data_collator=data_collator,
)

trainer_2.train()


Epoch,Training Loss,Validation Loss,Accuracy
1,0.359500,0.398955,0.882859
2,0.165700,0.559776,0.883234


Epoch,Training Loss,Validation Loss,Accuracy
1,0.359500,0.398955,0.882859
2,0.165700,0.559776,0.883234
3,0.084100,0.677553,0.879865


TrainOutput(global_step=7458, training_loss=0.20310397568833383, metrics={'train_runtime': 5064.4802, 'train_samples_per_second': 23.558, 'train_steps_per_second': 1.473, 'total_flos': 1.56958900075008e+16, 'train_loss': 0.20310397568833383, 'epoch': 3.0})

In [ ]:
trainer_eval = Trainer(
    model=model,
    eval_dataset=tokenized_test_dataset2,
    compute_metrics=compute_metrics_f1,

)

trainer_eval.evaluate()

{'eval_loss': 0.3940124809741974,
 'eval_precision': 0.6477987421383647,
 'eval_recall': 0.6369257950530035,
 'eval_f1': 0.6423162583518931,
 'eval_runtime': 104.0956,
 'eval_samples_per_second': 64.172,
 'eval_steps_per_second': 8.021}

# Predict

In [ ]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
pipe("Quang bị ngu à ")

In [ ]:
model.num_parameters()

115355906

In [ ]:
#load model
from transformers import AutoModel

# Đường dẫn tới thư mục chứa mô hình
model_path = "./results1"

# Tải mô hình
model = AutoModelForSequenceClassification.from_pretrained(model_path)

OSError: ./results1 does not appear to have a file named config.json. Checkout 'https://huggingface.co/./results1/tree/None' for available files.